# Brick optimisation - simple OOP test

Created 26/03/2025

Objectives:
* Previous methods working, but code getting messy so refactored into OOP solution. Test here.

# Package imports

In [1]:
import sys
sys.path.append('../')

In [2]:
import h5py
from tenpy.tools import hdf5_io
import tenpy
import tenpy.linalg.np_conserved as npc

import os

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [4]:
from SPTOptimization.SymmetryActionWithBoundaryUnitaries import SymmetryActionWithBoundaryUnitaries

from SPTOptimization.utils import (
    to_npc_array,
    get_right_identity_environment,
    get_right_identity_environment_from_tp_tensor,
    multiply_stacked_unitaries_against_mps,
    inner_product_b_tensors
)

from SPTOptimization.Optimizers.MPSBrickSolver import(
    MPSBrickSolver
)

# Load data

In [5]:
DATA_DIR = r"../data/simple_fermionic_cluster_200_site_dmrg/"

In [6]:
def parse_file_name(file_name):
    non_trivial_proj_rep = bool(int(file_name[3]))

    n1_01 = int(file_name[8])
    n1_10 = int(file_name[9])

    return (non_trivial_proj_rep, (n1_01, n1_10))

In [7]:
loaded_data = dict()

for local_file_name in os.listdir(DATA_DIR):
    f_name = r"{}/{}".format(DATA_DIR, local_file_name, ignore_unknown=False)
    with h5py.File(f_name, 'r') as f:
        data = hdf5_io.load_from_hdf5(f)

        data_info = parse_file_name(local_file_name)
        loaded_data[data_info]=data['wavefunction']

In [8]:
loaded_data

{(False, (1, 1)): <tenpy.networks.mps.MPS at 0x143d0a550>,
 (True, (1, 1)): <tenpy.networks.mps.MPS at 0x145a97b10>,
 (True, (0, 1)): <tenpy.networks.mps.MPS at 0x145af77d0>,
 (False, (0, 1)): <tenpy.networks.mps.MPS at 0x144bc05d0>,
 (False, (1, 0)): <tenpy.networks.mps.MPS at 0x150304f10>,
 (True, (1, 0)): <tenpy.networks.mps.MPS at 0x144bc1ed0>,
 (True, (0, 0)): <tenpy.networks.mps.MPS at 0x1503ce810>,
 (False, (0, 0)): <tenpy.networks.mps.MPS at 0x150406dd0>}

In [9]:
loaded_data[(True, (0,0))].expectation_value('N', sites=list(range(1, 201, 2)))

array([0.00000000e+00, 4.10310779e-55, 3.20108804e-32, 4.41166527e-30,
       4.30652209e-31, 1.28028128e-32, 2.18979328e-32, 9.64766348e-32,
       6.39204896e-31, 4.19476267e-32, 7.51835939e-33, 1.56998498e-32,
       3.06132263e-29, 6.24832887e-32, 6.64529320e-32, 6.61192056e-33,
       3.02331734e-32, 1.97077344e-32, 3.97848871e-32, 1.65936555e-32,
       8.12962747e-32, 8.36180681e-32, 1.05455111e-32, 2.30695596e-32,
       4.15719467e-32, 3.32855567e-32, 1.02762007e-32, 1.46697913e-32,
       3.80115562e-31, 1.63756131e-32, 9.98608231e-33, 1.82981182e-32,
       1.73380216e-33, 6.00451123e-32, 1.33036658e-32, 2.79091135e-31,
       1.64291033e-32, 3.68006128e-33, 9.95622775e-32, 6.17421509e-32,
       2.70280235e-32, 3.01371615e-31, 2.35183074e-32, 1.17896355e-32,
       3.93686429e-31, 8.45721273e-32, 1.08863312e-32, 3.77353007e-31,
       9.67562785e-33, 1.99656454e-33, 6.40393616e-33, 2.78084102e-32,
       1.03150150e-31, 4.37579428e-33, 1.45445289e-32, 3.55938597e-32,
      

In [10]:
loaded_data[(False, (0,0))].expectation_value('N', sites=list(range(1, 201, 2)))

array([0.00000000e+00, 3.26915052e-29, 3.33293732e-29, 0.00000000e+00,
       3.33293732e-29, 7.79030959e-29, 4.21824880e-29, 4.21824880e-29,
       5.96576060e-30, 1.14662165e-29, 3.59424750e-29, 8.18720523e-29,
       4.21824880e-29, 8.28796989e-29, 4.21824880e-29, 3.79670126e-29,
       4.21824880e-29, 8.18720523e-29, 1.37190923e-28, 0.00000000e+00,
       1.37190923e-28, 0.00000000e+00, 3.66111579e-29, 3.66111579e-29,
       3.86541844e-29, 3.33293732e-29, 0.00000000e+00, 0.00000000e+00,
       3.59424750e-29, 1.56000325e-28, 3.46235982e-29, 0.00000000e+00,
       3.59424750e-29, 3.59424750e-29, 3.72860037e-29, 0.00000000e+00,
       0.00000000e+00, 9.22258517e-29, 8.33234331e-30, 9.01057880e-29,
       3.46235982e-29, 4.43734259e-29, 3.14342582e-29, 3.14342582e-29,
       0.00000000e+00, 8.18720523e-29, 1.46709639e-29, 1.46709639e-29,
       1.46709639e-29, 8.98561875e-30, 0.00000000e+00, 8.01495006e-30,
       4.07526776e-29, 4.00470169e-29, 9.32150093e-30, 7.69262642e-29,
      

In [11]:
loaded_exps = dict()

for k, psi in loaded_data.items():
    exp = psi.expectation_value('N', sites=[101,])[0]
    loaded_exps[k] = exp

In [12]:
loaded_exps

{(False, (1, 1)): 0.4999999999999988,
 (True, (1, 1)): 0.4999999999999729,
 (True, (0, 1)): 0.49999999999999745,
 (False, (0, 1)): 0.4999999999999949,
 (False, (1, 0)): 0.49999999999998695,
 (True, (1, 0)): 0.49999999999999367,
 (True, (0, 0)): 6.403936160073908e-33,
 (False, (0, 0)): 0.0}

# Definitons

In [13]:
MAX_VIRTUAL_BOND_DIM = 8
MAX_INTERMEDIATE_VIRTUAL_BOND_DIM = 2*MAX_VIRTUAL_BOND_DIM
# MPO bond dim?
MAX_MPO_BOND_DIM = 50

SVD_CUTOFF = 1e-3

Define bosonic symmetries. Label by the group element added.

In [14]:
np_00 = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
])

np_01 = np.array([
    [0, 1, 0, 0],
    [1, 0, 0, 0],
    [0, 0, 0, 1],
    [0, 0, 1, 0]
])

np_10 = np.array([
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [1, 0, 0, 0],
    [0, 1, 0, 0]
])

np_11 = np.array([
    [0, 0, 0, 1],
    [0, 0, 1, 0],
    [0, 1, 0, 0],
    [1, 0, 0, 0]
])

In [15]:
bosonic_np_symmetries = [
    np_00,
    np_01,
    np_10,
    np_11
]

In [16]:
npc_00 = to_npc_array(np_00)
npc_01 = to_npc_array(np_01)
npc_10 = to_npc_array(np_10)
npc_11 = to_npc_array(np_11)

Define "fermionic symmetries". Just identity and JW string.

In [17]:
np_I = np.array([
    [1, 0],
    [0, 1]
])

np_JW = np.array([
    [1, 0],
    [0, -1]
])

In [18]:
fermionic_np_symmetries = [np_I, np_JW]

In [19]:
fermionic_npc_symmetries = [
    to_npc_array(X) for X in fermionic_np_symmetries
]

In [20]:
npc_JW = fermionic_npc_symmetries[1]

In [21]:
symmetry_actions = [
    [[b, f] for b in bosonic_np_symmetries]
    for f in fermionic_np_symmetries
]

In [22]:
shifted_symmetry_actions = [
    [[f, b] for b in bosonic_np_symmetries]
    for f in fermionic_np_symmetries
]

In [23]:
cases = dict()

for k, psi in loaded_data.items():

    for i, l in enumerate(symmetry_actions):

        for j, s in enumerate(l):
            case = SymmetryActionWithBoundaryUnitaries(
                psi,
                s*50,
                left_symmetry_index=50,
                left_boundary_unitaries=[np_I, np_00]*1,
                right_boundary_unitaries=[np_00, np_I]*1
            )

            cases[(*k, i, j)] = case

In [24]:
for c in cases.values():
    c.compute_svd_approximate_expectation()

In [25]:
shifted_cases = dict()

for k, psi in loaded_data.items():

    for i, l in enumerate(shifted_symmetry_actions):

        for j, s in enumerate(l):
            case = SymmetryActionWithBoundaryUnitaries(
                psi,
                s*50,
                left_symmetry_index=51,
                left_boundary_unitaries=[np_00, np_I]*1,
                right_boundary_unitaries=[np_I, np_00]*1
            )

            shifted_cases[(*k, i, j)] = case

In [26]:
for c in shifted_cases.values():
    c.compute_svd_approximate_expectation()

In [27]:
symmetric_cases = dict()

for k, psi in loaded_data.items():

    for i, l in enumerate(symmetry_actions):

        for j, s in enumerate(l):
            case = SymmetryActionWithBoundaryUnitaries(
                psi,
                [s[1],] + s*49,
                left_symmetry_index=51,
                left_boundary_unitaries=[np_00, np_I, np_00],
                right_boundary_unitaries=[np_00, np_I, np_00]
            )

            symmetric_cases[(*k, i, j)] = case

In [28]:
for c in symmetric_cases.values():
    c.compute_svd_approximate_expectation()

In [29]:
left_trivial_leg_charge = tenpy.linalg.charges.LegCharge(
    tenpy.linalg.charges.ChargeInfo([], []),
    [0,1],
    [[]],
    qconj=1
)

In [30]:
right_trivial_leg_charge = tenpy.linalg.charges.LegCharge(
    tenpy.linalg.charges.ChargeInfo([], []),
    [0,1],
    [[]],
    qconj=-1
)

# Test case

In [31]:
test_case = cases[False, (1,0), 1, 1]

In [32]:
block_width = 3
num_one_sided_iterations = 3
num_two_sided_iterations = 3
num_layers = 1
num_sites = 3

In [33]:
test_opt = MPSBrickSolver(
    test_case,
    num_sites,
    block_width,
    num_layers,
    num_one_sided_iterations,
    num_two_sided_iterations,
    max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM
)

In [34]:
test_opt.optimise()

In [35]:
test_opt.right_expectations

[[[[1.000000000000003], [1.000000000000003], [1.000000000000003]],
  [[1.0000000000000022], [1.0000000000000022], [1.0000000000000022]],
  [[1.0000000000000036], [1.0000000000000036], [1.0000000000000036]]]]

In [36]:
test_opt.left_expectations

[[[[3.079773827816547e-17], [3.079773827816547e-17], [3.079773827816547e-17]],
  [[1.0000000000000027], [1.0000000000000027], [1.0000000000000027]],
  [[1.0000000000000029], [1.0000000000000029], [1.0000000000000029]]]]

In [37]:
test_opt.right_unitaries

[[<npc.Array shape=(32, 32) labels=['(p0.p1.p2)', '(p0*.p1*.p2*)']>]]

In [38]:
test_opt.left_unitaries

[[<npc.Array shape=(16, 16) labels=['(p0.p1.p2)', '(p0*.p1*.p2*)']>]]

# Sweep

In [39]:
block_width = 3
num_one_sided_iterations = 3
num_two_sided_iterations = 3
num_layers = 1
num_sites = 3

In [40]:
optimizer_dict = dict()

for k, v in cases.items():
    opt = MPSBrickSolver(
        v,
        num_sites,
        block_width,
        num_layers,
        num_one_sided_iterations,
        num_two_sided_iterations,
        max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM
    )

    opt.optimise()

    optimizer_dict[k] = opt

In [41]:
final_exps = dict()

for k, v in optimizer_dict.items():
    final_exp = v.flatten_exps()[-1]
    final_exps[k] = final_exp

In [42]:
bad_cases = [
    k
    for k, v in final_exps.items()
    if v < 0.9
]

In [43]:
bad_cases

[(True, (1, 1), 0, 1),
 (True, (1, 1), 0, 2),
 (True, (1, 1), 1, 1),
 (True, (1, 1), 1, 2),
 (True, (0, 1), 0, 2),
 (True, (0, 1), 0, 3),
 (True, (0, 1), 1, 2),
 (True, (0, 1), 1, 3),
 (True, (1, 0), 0, 1),
 (True, (1, 0), 0, 3),
 (True, (1, 0), 1, 1),
 (True, (1, 0), 1, 3),
 (True, (0, 0), 0, 1),
 (True, (0, 0), 0, 2),
 (True, (0, 0), 0, 3),
 (True, (0, 0), 1, 1),
 (True, (0, 0), 1, 2),
 (True, (0, 0), 1, 3)]

In [44]:
len(bad_cases)

18

Try again with wider parameters.

In [45]:
block_width = 5
num_one_sided_iterations = 1
num_two_sided_iterations = 3
num_layers = 1
num_sites = 5

In [46]:
optimizer_dict = dict()

for k, v in cases.items():
    opt = MPSBrickSolver(
        v,
        num_sites,
        block_width,
        num_layers,
        num_one_sided_iterations,
        num_two_sided_iterations,
        max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM
    )

    opt.optimise()

    optimizer_dict[k] = opt

In [47]:
final_exps = dict()

for k, v in optimizer_dict.items():
    final_exp = v.flatten_exps()[-1]
    final_exps[k] = final_exp

In [48]:
bad_cases = [
    k
    for k, v in final_exps.items()
    if v < 0.99
]

In [49]:
bad_cases

[]

In [50]:
final_exps

{(False, (1, 1), 0, 0): 1.0000000000000075,
 (False, (1, 1), 0, 1): 1.0000000000000073,
 (False, (1, 1), 0, 2): 1.0000000000000082,
 (False, (1, 1), 0, 3): 1.0000000000000082,
 (False, (1, 1), 1, 0): 1.0000000000000078,
 (False, (1, 1), 1, 1): 1.0000000000000089,
 (False, (1, 1), 1, 2): 1.0000000000000073,
 (False, (1, 1), 1, 3): 1.0000000000000078,
 (True, (1, 1), 0, 0): 1.0000000000000144,
 (True, (1, 1), 0, 1): 1.0000000000000142,
 (True, (1, 1), 0, 2): 1.0000000000000149,
 (True, (1, 1), 0, 3): 1.0000000000000147,
 (True, (1, 1), 1, 0): 1.000000000000014,
 (True, (1, 1), 1, 1): 1.000000000000015,
 (True, (1, 1), 1, 2): 1.0000000000000149,
 (True, (1, 1), 1, 3): 1.0000000000000144,
 (True, (0, 1), 0, 0): 1.0000000000000149,
 (True, (0, 1), 0, 1): 1.0000000000000142,
 (True, (0, 1), 0, 2): 1.0000000000000149,
 (True, (0, 1), 0, 3): 1.0000000000000135,
 (True, (0, 1), 1, 0): 1.0000000000000142,
 (True, (0, 1), 1, 1): 1.0000000000000138,
 (True, (0, 1), 1, 2): 1.0000000000000138,
 (Tru

Double check expectations.

In [51]:
(
    test_opt.left_side_environment,
    test_opt.right_side_environment,
    test_opt.right_side_left_symmetry_environment,
    test_opt.left_side_right_symmetry_environment
)

(<npc.Array shape=(2, 2) labels=['vR', 'vR*']>,
 <npc.Array shape=(2, 2) labels=['vR', 'vR*']>,
 <npc.Array shape=(2, 2) labels=['vR', 'vR*']>,
 <npc.Array shape=(2, 2) labels=['vR*', 'vR']>)

In [52]:
def check_expectations(opt):
    e1 = npc.tensordot(
        opt.left_side_environment,
        opt.right_side_left_symmetry_environment,
        [['vR', 'vR*'], ['vR', 'vR*']]
    )

    e2 = npc.tensordot(
        opt.right_side_environment,
        opt.left_side_right_symmetry_environment,
        [['vR', 'vR*'] ,['vR', 'vR*']]
    )

    return (e1, e2)

In [53]:
check_exps = {
    k: check_expectations(v)
    for k, v in optimizer_dict.items()
}

In [54]:
check_exps

{(False, (1, 1), 0, 0): ((1.0000000000000002+0j), (1.0000000000000002+0j)),
 (False, (1, 1), 0, 1): ((1.0000000000000009+0j), (1.0000000000000009+0j)),
 (False, (1, 1), 0, 2): ((1.0000000000000018+0j), (1.0000000000000016+0j)),
 (False, (1, 1), 0, 3): ((1.000000000000001+0j), (1.000000000000001+0j)),
 (False, (1, 1), 1, 0): ((1.0000000000000018+0j), (1.0000000000000018+0j)),
 (False, (1, 1), 1, 1): ((1.000000000000002+0j), (1.000000000000002+0j)),
 (False, (1, 1), 1, 2): ((1.000000000000001+0j), (1.000000000000001+0j)),
 (False, (1, 1), 1, 3): ((1.0000000000000013+0j), (1.0000000000000013+0j)),
 (True, (1, 1), 0, 0): ((1.0000000000000129+0j), (1.0000000000000127+0j)),
 (True, (1, 1), 0, 1): ((1.0000000000000129+0j), (1.0000000000000129+0j)),
 (True, (1, 1), 0, 2): ((1.0000000000000138+0j), (1.0000000000000138+0j)),
 (True, (1, 1), 0, 3): ((1.0000000000000129+0j), (1.0000000000000127+0j)),
 (True, (1, 1), 1, 0): ((1.0000000000000124+0j), (1.0000000000000127+0j)),
 (True, (1, 1), 1, 1): 

In [55]:
final_exps

{(False, (1, 1), 0, 0): 1.0000000000000075,
 (False, (1, 1), 0, 1): 1.0000000000000073,
 (False, (1, 1), 0, 2): 1.0000000000000082,
 (False, (1, 1), 0, 3): 1.0000000000000082,
 (False, (1, 1), 1, 0): 1.0000000000000078,
 (False, (1, 1), 1, 1): 1.0000000000000089,
 (False, (1, 1), 1, 2): 1.0000000000000073,
 (False, (1, 1), 1, 3): 1.0000000000000078,
 (True, (1, 1), 0, 0): 1.0000000000000144,
 (True, (1, 1), 0, 1): 1.0000000000000142,
 (True, (1, 1), 0, 2): 1.0000000000000149,
 (True, (1, 1), 0, 3): 1.0000000000000147,
 (True, (1, 1), 1, 0): 1.000000000000014,
 (True, (1, 1), 1, 1): 1.000000000000015,
 (True, (1, 1), 1, 2): 1.0000000000000149,
 (True, (1, 1), 1, 3): 1.0000000000000144,
 (True, (0, 1), 0, 0): 1.0000000000000149,
 (True, (0, 1), 0, 1): 1.0000000000000142,
 (True, (0, 1), 0, 2): 1.0000000000000149,
 (True, (0, 1), 0, 3): 1.0000000000000135,
 (True, (0, 1), 1, 0): 1.0000000000000142,
 (True, (0, 1), 1, 1): 1.0000000000000138,
 (True, (0, 1), 1, 2): 1.0000000000000138,
 (Tru

In [56]:
test_opt = optimizer_dict[(True, (1, 0), 1, 1)]

In [57]:
base_bs = test_opt.bottom_right_mps_tensors

In [58]:
base_bs

[<npc.Array shape=(4, 4, 4) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(4, 2, 4) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(4, 4, 4) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(4, 2, 4) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(4, 4, 4) labels=['vL', 'p', 'vR']>]

In [59]:
npc_JW

<npc.Array shape=(2, 2) labels=['p', 'p*']>

In [60]:
fp_bs = list()

for i, b in enumerate(base_bs):
    # Need to update this logic
    if i%2:
        b_to_add = npc.tensordot(b, npc_JW, [['p',], ['p*',]])
    else:
        b_to_add = b

    fp_bs.append(b_to_add)

In [61]:
fp_o_bs, _ = multiply_stacked_unitaries_against_mps(
    test_opt.right_unitaries,
    fp_bs,
    test_opt.original_right_side_left_schmidt_values,
    MAX_VIRTUAL_BOND_DIM
)

In [62]:
inner_product_b_tensors(
    fp_o_bs,
    fp_bs,
    left_environment=test_opt.right_side_left_symmetry_environment
)

(-1.0000000000000142+0j)

Sweep over solutions, make into a function.

In [63]:
l = list(range(10))

In [64]:
l[-2::-1]

[8, 7, 6, 5, 4, 3, 2, 1, 0]

In [65]:
def right_fp_charge(optimizer):
    opt = optimizer

    base_bs = opt.bottom_right_mps_tensors

    fp_bs = list()
    
    for i, b in enumerate(base_bs):
        # Need to update this logic
        if i%2:
            b_to_add = npc.tensordot(b, npc_JW, [['p',], ['p*',]])
        else:
            b_to_add = b
    
        fp_bs.append(b_to_add)

    fp_o_bs, _ = multiply_stacked_unitaries_against_mps(
        opt.right_unitaries,
        fp_bs,
        opt.original_right_side_left_schmidt_values,
        opt.max_virtual_bond_dim
    )

    out = inner_product_b_tensors(
        fp_o_bs,
        fp_bs,
        left_environment=opt.right_side_left_symmetry_environment
    )

    return out

In [66]:
right_fp_charges = dict()

for k, v in optimizer_dict.items():
    right_fp_charges[k] = right_fp_charge(v)

In [67]:
right_fp_charges

{(False, (1, 1), 0, 0): (1.0000000000000004+0j),
 (False, (1, 1), 0, 1): (-1.0000000000000002+0j),
 (False, (1, 1), 0, 2): (-1.0000000000000009+0j),
 (False, (1, 1), 0, 3): (1.000000000000001+0j),
 (False, (1, 1), 1, 0): (1.0000000000000018+0j),
 (False, (1, 1), 1, 1): (-1.000000000000002+0j),
 (False, (1, 1), 1, 2): (-1.0000000000000018+0j),
 (False, (1, 1), 1, 3): (1.0000000000000013+0j),
 (True, (1, 1), 0, 0): (1.0000000000000129+0j),
 (True, (1, 1), 0, 1): (-1.0000000000000129+0j),
 (True, (1, 1), 0, 2): (-1.0000000000000133+0j),
 (True, (1, 1), 0, 3): (1.0000000000000127+0j),
 (True, (1, 1), 1, 0): (1.0000000000000122+0j),
 (True, (1, 1), 1, 1): (-1.0000000000000138+0j),
 (True, (1, 1), 1, 2): (-1.000000000000013+0j),
 (True, (1, 1), 1, 3): (1.0000000000000129+0j),
 (True, (0, 1), 0, 0): (1.0000000000000142+0j),
 (True, (0, 1), 0, 1): (1.0000000000000133+0j),
 (True, (0, 1), 0, 2): (-1.0000000000000135+0j),
 (True, (0, 1), 0, 3): (-1.0000000000000124+0j),
 (True, (0, 1), 1, 0): (1

# Conclusion
It works!

# Old code

In [68]:
def right_fp_charge_contraction(optimizer):
    opt = optimizer

    base_bs = opt.bottom_right_mps_tensors

    fp_bs = list()
    
    for i, b in enumerate(base_bs):
        # Need to update this logic
        if i%2:
            b_to_add = npc.tensordot(b, npc_JW, [['p',], ['p*',]])
        else:
            b_to_add = b
    
        fp_bs.append(b_to_add)

    unitary = opt.right_unitaries[0][0].split_legs()

    t = unitary

    t = npc.tensordot(
        t,
        fp_bs[0],
        [['p0*',], ['p']]
    )

    t = npc.tensordot(
        t,
        fp_bs[0].conj(),
        [['p0',], ['p*']]
    )

    t = npc.tensordot(
        t,
        opt.right_side_left_symmetry_environment,
        [['vL', 'vL*',], ['vR', 'vR*']]
    )

    for i, b in enumerate(fp_bs[1:], start=1):
        t = npc.tensordot(
            t,
            b,
            [[f'p{i}*', 'vR'], ['p', 'vL']]
        )
    
        t = npc.tensordot(
            t,
            b.conj(),
            [[f'p{i}', 'vR*'], ['p*', 'vL*']]
        )
    
    out1 = npc.trace(t, 'vR', 'vR*')

    t = unitary

    t = npc.tensordot(
        t,
        base_bs[0],
        [['p0*',], ['p']]
    )

    t = npc.tensordot(
        t,
        base_bs[0].conj(),
        [['p0',], ['p*']]
    )

    t = npc.tensordot(
        t,
        opt.right_side_left_symmetry_environment,
        [['vL', 'vL*',], ['vR', 'vR*']]
    )

    for i, b in enumerate(base_bs[1:], start=1):
        t = npc.tensordot(
            t,
            b,
            [[f'p{i}*', 'vR'], ['p', 'vL']]
        )
    
        t = npc.tensordot(
            t,
            b.conj(),
            [[f'p{i}', 'vR*'], ['p*', 'vL*']]
        )
    
    out2 = npc.trace(t, 'vR', 'vR*')

    return (out1, out2)

In [69]:
right_fp_charge_contraction(test_opt)

((-1.0000000000000147+0j), (1.0000000000000147+0j))

In [70]:
right_fp_contraction_charges = dict()

for k, v in optimizer_dict.items():
    right_fp_contraction_charges[k] = right_fp_charge_contraction(v)

In [71]:
right_fp_contraction_charges

{(False, (1, 1), 0, 0): ((1.0000000000000004+0j), (1.0000000000000004+0j)),
 (False, (1, 1), 0, 1): ((-1.0000000000000002+0j), (1.0000000000000002+0j)),
 (False, (1, 1), 0, 2): ((-1.000000000000001+0j), (1.000000000000001+0j)),
 (False, (1, 1), 0, 3): ((1.0000000000000009+0j), (1.0000000000000009+0j)),
 (False, (1, 1), 1, 0): ((1.0000000000000013+0j), (1.000000000000001+0j)),
 (False, (1, 1), 1, 1): ((-1.000000000000002+0j), (1.0000000000000018+0j)),
 (False, (1, 1), 1, 2): ((-1.0000000000000007+0j), (1.0000000000000007+0j)),
 (False, (1, 1), 1, 3): ((1.0000000000000009+0j), (1.0000000000000009+0j)),
 (True, (1, 1), 0, 0): ((1.0000000000000138+0j), (1.0000000000000135+0j)),
 (True, (1, 1), 0, 1): ((-1.0000000000000133+0j), (1.0000000000000133+0j)),
 (True, (1, 1), 0, 2): ((-1.0000000000000138+0j), (1.000000000000014+0j)),
 (True, (1, 1), 0, 3): ((1.000000000000014+0j), (1.000000000000014+0j)),
 (True, (1, 1), 1, 0): ((1.0000000000000127+0j), (1.0000000000000129+0j)),
 (True, (1, 1), 1,

In [72]:
diffs = list()

for k in right_fp_contraction_charges:
    c1 = right_fp_charges[k]
    c2 = right_fp_contraction_charges[k]
    diff = np.abs(c1-c2)
    diffs.append(diff)

diffs = np.array(diffs)

In [73]:
np.max(diffs)

2.000000000000029

So they're the same...

Try old method.

In [74]:
def extract_right_rdm(case, num_sites):
    t = case.right_projected_symmetry_state
    psi = case.psi

    right_symmetry_index = case.right_symmetry_index + 1
    sites = range(right_symmetry_index, right_symmetry_index + num_sites)
    for i, site_index in enumerate(sites):
        b = psi.get_B(site_index)
        b = b.replace_label('p', f'p{i}')
        
        t = npc.tensordot(t, b, [['vR',], ['vL',]])
        t = npc.tensordot(t, b.conj(), [['vR*',], ['vL*',]])
    
    t = npc.trace(t, 'vR', 'vR*')

    sing_val = case.symmetry_transfer_matrix_singular_vals[0]
    t*= np.sqrt(sing_val)

    return t

In [75]:
right_rdms = dict()

for k, v in optimizer_dict.items():
    right_rdms[k] = extract_right_rdm(v.symmetry_case, 5)

In [76]:
np_fp_operator = (
    np_00[(...,) + (np.newaxis,)*8]
    * np_JW[(np.newaxis,)*2 + (...,) + (np.newaxis,)*6]
    * np_00[(np.newaxis,)*4  + (...,) + (np.newaxis,)*4]
    * np_JW[(np.newaxis,)*6 + (...,) + (np.newaxis,)*2]
    * np_00[(np.newaxis,)*8 + (...,)]
)

In [77]:
fp_operator = npc.Array.from_ndarray_trivial(
    np_fp_operator,
    labels = [
        'p0',
        'p0*',
        'p1',
        'p1*',
        'p2',
        'p2*',
        'p3',
        'p3*',
        'p4',
        'p4*'
    ]
)

In [78]:
def fp_operator_check(rdm, svd_sol):
    t = rdm

    t = npc.tensordot(
        t,
        fp_operator,
        [
            ['p0', 'p1', 'p2', 'p3', 'p4'],
            ['p0*', 'p1*', 'p2*', 'p3*', 'p4*']
        ]
    )

    t = npc.tensordot(
        t,
        svd_sol.split_legs(),
        [
            ['p0', 'p1', 'p2', 'p3', 'p4'],
            ['p0*', 'p1*', 'p2*', 'p3*', 'p4*']
        ]
    )

    out1 = npc.tensordot(
        t,
        fp_operator,
        [
            ['p0', 'p1', 'p2', 'p3', 'p4', 'p0*', 'p1*', 'p2*', 'p3*', 'p4*'],
            ['p0*', 'p1*', 'p2*', 'p3*', 'p4*', 'p0', 'p1', 'p2', 'p3', 'p4']
        ]
    )

    out2 = npc.tensordot(
        rdm,
        svd_sol.split_legs(),
        [
            ['p0', 'p1', 'p2', 'p3', 'p4', 'p0*', 'p1*', 'p2*', 'p3*', 'p4*'],
            ['p0*', 'p1*', 'p2*', 'p3*', 'p4*', 'p0', 'p1', 'p2', 'p3', 'p4']
        ]
    )

    
    return (out1, out2)

In [79]:
fp_rdm_checks = dict()

for k, v in right_rdms.items():
    unitary = optimizer_dict[k].right_unitaries[0][0]
    out = fp_operator_check(v, unitary)

    fp_rdm_checks[k] = out

In [80]:
fp_rdm_checks

{(False, (1, 1), 0, 0): ((-1.0000000000000002+0j), (-1.0000000000000002+0j)),
 (False, (1, 1), 0, 1): ((0.9999999999999998+0j), (-0.9999999999999998+0j)),
 (False, (1, 1), 0, 2): ((1+0j), (-1.0000000000000002+0j)),
 (False, (1, 1), 0, 3): ((-1+0j), (-1+0j)),
 (False, (1, 1), 1, 0): ((-0.9999999999999999+0j), (-0.9999999999999996+0j)),
 (False, (1, 1), 1, 1): ((0.9999999999999998+0j), (-0.9999999999999998+0j)),
 (False, (1, 1), 1, 2): ((1+0j), (-0.9999999999999999+0j)),
 (False, (1, 1), 1, 3): ((-0.9999999999999999+0j), (-0.9999999999999999+0j)),
 (True, (1, 1), 0, 0): ((-1.0000000000000075+0j), (-1.0000000000000075+0j)),
 (True, (1, 1), 0, 1): ((-1.0000000000000073+0j), (1.0000000000000073+0j)),
 (True, (1, 1), 0, 2): ((-1.0000000000000073+0j), (1.000000000000007+0j)),
 (True, (1, 1), 0, 3): ((-1.0000000000000075+0j), (-1.0000000000000075+0j)),
 (True, (1, 1), 1, 0): ((-1.000000000000007+0j), (-1.0000000000000073+0j)),
 (True, (1, 1), 1, 1): ((1.0000000000000069+0j), (-1.00000000000000

Try old solution method. (New one appears to not be working...!)

In [81]:
def extract_SVD(rdm):
    grouped_rdm = rdm.combine_legs([
        ['p0', 'p1', 'p2', 'p3', 'p4'],
        ['p0*', 'p1*', 'p2*', 'p3*', 'p4*']
    ])

    U, S, VH = npc.svd(grouped_rdm, inner_labels=['i', 'i*'])

    out = npc.tensordot(U, VH, [['i',], ['i*',]])
    out = out.conj()

    test = npc.tensordot(
        grouped_rdm,
        out,
        [
            ['(p0.p1.p2.p3.p4)', '(p0*.p1*.p2*.p3*.p4*)'],
            ['(p0*.p1*.p2*.p3*.p4*)', '(p0.p1.p2.p3.p4)']
        ]
    )

    return np.sum(S), out, test

In [82]:
X = (
    test_opt.right_side_left_symmetry_environment.to_ndarray()
    - test_opt.symmetry_case.right_projected_symmetry_state.to_ndarray()
)

In [83]:
np.round(X, 2)

array([[-0.  +0.j,  0.05+0.j, -0.19+0.j, -0.15+0.j],
       [-0.05+0.j, -0.  +0.j, -0.15+0.j,  0.19+0.j],
       [ 0.19+0.j,  0.15+0.j,  0.  +0.j,  0.05+0.j],
       [ 0.15+0.j, -0.19+0.j, -0.05+0.j,  0.  +0.j]])

In [84]:
test_opt.symmetry_case.symmetry_transfer_matrix_singular_vals

array([2.50000000e-01, 2.20074957e-17, 1.59584405e-17, 8.09078008e-18,
       6.10133891e-18, 3.60538764e-18, 2.86876835e-18, 1.30596327e-18,
       5.83134352e-19, 3.79361049e-19, 4.77755246e-34, 2.75125670e-34,
       1.40409237e-34, 1.95609703e-35, 7.97635267e-36, 6.02520616e-38])

Non-degenerate, so the left/right environment should _not_ be changing appreciably...!

In [85]:
l = list(range(10))

In [86]:
l[::-1]

[9, 8, 7, 6, 5, 4, 3, 2, 1, 0]

In [87]:
l

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [88]:
X = (
    test_opt.left_side_right_symmetry_environment.to_ndarray()
    - test_opt.symmetry_case.left_projected_symmetry_state.to_ndarray()
)

In [89]:
np.round(X, 2)

array([[-0.  +0.j, -0.18+0.j,  0.02+0.j,  0.73+0.j],
       [ 0.18+0.j,  0.  +0.j, -0.73+0.j,  0.02+0.j],
       [-0.02+0.j,  0.73+0.j,  0.  +0.j,  0.18+0.j],
       [-0.73+0.j, -0.02+0.j, -0.18+0.j, -0.  +0.j]])

In [90]:
X = (
    test_opt.right_side_left_symmetry_environment.to_ndarray()
    - test_opt.symmetry_case.left_projected_symmetry_state.to_ndarray()
)

In [91]:
np.round(X, 2)

array([[-0.  +0.j, -0.17+0.j,  0.2 +0.j,  0.64+0.j],
       [ 0.17+0.j,  0.  +0.j, -0.33+0.j, -0.18+0.j],
       [-0.2 +0.j,  0.33+0.j, -0.  +0.j,  0.06+0.j],
       [-0.64+0.j,  0.18+0.j, -0.06+0.j, -0.  +0.j]])

Environments are wrong, so just do it the "dumb" way for now.

In [92]:
right_svd_solds = {
    k: extract_SVD(v) for k, v in right_rdms.items()
}

In [93]:
{k: (v[0], v[-1]) for k, v in right_svd_solds.items()}

{(False, (1, 1), 0, 0): (1.0000000000000084, (1.0000000000000004+0j)),
 (False, (1, 1), 0, 1): (1.0000000000000078, (1+0j)),
 (False, (1, 1), 0, 2): (1.000000000000008, (1.0000000000000004+0j)),
 (False, (1, 1), 0, 3): (1.0000000000000078, (1.0000000000000002+0j)),
 (False, (1, 1), 1, 0): (1.0000000000000087, (1.0000000000000004+0j)),
 (False, (1, 1), 1, 1): (1.000000000000008, (1+0j)),
 (False, (1, 1), 1, 2): (1.0000000000000082, (1.0000000000000002+0j)),
 (False, (1, 1), 1, 3): (1.000000000000008, (1+0j)),
 (True, (1, 1), 0, 0): (1.0000000000000087, (1.0000000000000075+0j)),
 (True, (1, 1), 0, 1): (1.0000000000000089, (1.0000000000000075+0j)),
 (True, (1, 1), 0, 2): (1.0000000000000084, (1.0000000000000073+0j)),
 (True, (1, 1), 0, 3): (1.0000000000000084, (1.0000000000000078+0j)),
 (True, (1, 1), 1, 0): (1.0000000000000089, (1.0000000000000075+0j)),
 (True, (1, 1), 1, 1): (1.0000000000000082, (1.0000000000000073+0j)),
 (True, (1, 1), 1, 2): (1.0000000000000082, (1.000000000000007+0j)

In [94]:
right_svd_fp_checks = {
    k: fp_operator_check(right_rdms[k], right_svd_solds[k][1])
    for k in right_svd_solds
}

In [95]:
right_svd_fp_checks

{(False, (1, 1), 0, 0): ((1.0000000000000002+0j), (1.0000000000000004+0j)),
 (False, (1, 1), 0, 1): ((-0.9999999999999996+0j), (1+0j)),
 (False, (1, 1), 0, 2): ((-1+0j), (1.0000000000000004+0j)),
 (False, (1, 1), 0, 3): ((1.0000000000000002+0j), (1.0000000000000002+0j)),
 (False, (1, 1), 1, 0): ((1.0000000000000004+0j), (1.0000000000000004+0j)),
 (False, (1, 1), 1, 1): ((-1+0j), (1+0j)),
 (False, (1, 1), 1, 2): ((-1+0j), (1.0000000000000002+0j)),
 (False, (1, 1), 1, 3): ((0.9999999999999998+0j), (1+0j)),
 (True, (1, 1), 0, 0): ((1.0000000000000073+0j), (1.0000000000000075+0j)),
 (True, (1, 1), 0, 1): ((-1.0000000000000075+0j), (1.0000000000000075+0j)),
 (True, (1, 1), 0, 2): ((-1.0000000000000073+0j), (1.0000000000000073+0j)),
 (True, (1, 1), 0, 3): ((1.0000000000000075+0j), (1.0000000000000078+0j)),
 (True, (1, 1), 1, 0): ((1.0000000000000073+0j), (1.0000000000000075+0j)),
 (True, (1, 1), 1, 1): ((-1.0000000000000073+0j), (1.0000000000000073+0j)),
 (True, (1, 1), 1, 2): ((-1.000000000

Debug environments.

In [96]:
test_case = cases[(True, (0, 0), 1, 3)]

In [97]:
block_width = 5
num_one_sided_iterations = 3
num_two_sided_iterations = 3
num_layers = 1
num_sites = 5

In [98]:
test_opt = MPSBrickSolver(
    test_case,
    num_sites,
    block_width,
    num_layers,
    num_one_sided_iterations,
    num_two_sided_iterations,
    max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM
)

In [99]:
np.round(test_opt.right_side_left_symmetry_environment.to_ndarray(), 2)

array([[-0.+0.j,  0.+0.j,  0.+0.j, -0.+0.j],
       [ 0.+0.j,  0.+0.j, -0.+0.j,  0.+0.j],
       [ 0.+0.j, -0.+0.j,  0.+0.j, -0.+0.j],
       [-0.+0.j,  0.+0.j, -0.+0.j, -0.+0.j]])

In [100]:
test_opt.symmetry_case.symmetry_transfer_matrix_singular_vals

array([2.50000000e-01, 3.79699847e-17, 2.05675553e-17, 1.85337541e-17,
       1.27243097e-17, 1.02241480e-17, 8.15437879e-18, 5.55094259e-18,
       4.83809810e-18, 2.34598487e-18, 1.95503610e-18, 1.01945860e-18,
       5.71980066e-19, 2.85902674e-19, 1.69509079e-19, 2.19433754e-34])

In [101]:
np.round(test_opt.symmetry_case.right_projected_symmetry_state.to_ndarray(), 2)

array([[-0.14+0.j,  0.27+0.j,  0.38+0.j, -0.08+0.j],
       [ 0.27+0.j,  0.38+0.j, -0.16+0.j,  0.06+0.j],
       [ 0.38+0.j, -0.16+0.j,  0.22+0.j, -0.17+0.j],
       [-0.08+0.j,  0.06+0.j, -0.17+0.j, -0.46+0.j]])

Looks comparable... but maybe too small to be sure?

In [102]:
test_opt.optimise()

In [103]:
np.round(test_opt.right_side_left_symmetry_environment.to_ndarray(), 2)

array([[-0.07+0.j,  0.14+0.j,  0.19+0.j, -0.04+0.j],
       [ 0.14+0.j,  0.19+0.j, -0.08+0.j,  0.03+0.j],
       [ 0.19+0.j, -0.08+0.j,  0.11+0.j, -0.09+0.j],
       [-0.04+0.j,  0.03+0.j, -0.09+0.j, -0.23+0.j]])

In [104]:
np.round(test_opt.symmetry_case.right_projected_symmetry_state.to_ndarray(), 2)

array([[-0.14+0.j,  0.27+0.j,  0.38+0.j, -0.08+0.j],
       [ 0.27+0.j,  0.38+0.j, -0.16+0.j,  0.06+0.j],
       [ 0.38+0.j, -0.16+0.j,  0.22+0.j, -0.17+0.j],
       [-0.08+0.j,  0.06+0.j, -0.17+0.j, -0.46+0.j]])

Totally different. Step through and debug.

In [105]:
test_case = cases[(True, (0, 0), 1, 3)]

In [106]:
block_width = 5
num_one_sided_iterations = 3
num_two_sided_iterations = 3
num_layers = 1
num_sites = 5

In [107]:
test_opt = MPSBrickSolver(
    test_case,
    num_sites,
    block_width,
    num_layers,
    num_one_sided_iterations,
    num_two_sided_iterations,
    max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM
)

In [108]:
np.round(test_opt.right_side_left_symmetry_environment.to_ndarray(), 2)

array([[-0.+0.j,  0.+0.j,  0.+0.j, -0.+0.j],
       [ 0.+0.j,  0.+0.j, -0.+0.j,  0.+0.j],
       [ 0.+0.j, -0.+0.j,  0.+0.j, -0.+0.j],
       [-0.+0.j,  0.+0.j, -0.+0.j, -0.+0.j]])

In [109]:
test_opt.symmetry_case.symmetry_transfer_matrix_singular_vals

array([2.50000000e-01, 3.79699847e-17, 2.05675553e-17, 1.85337541e-17,
       1.27243097e-17, 1.02241480e-17, 8.15437879e-18, 5.55094259e-18,
       4.83809810e-18, 2.34598487e-18, 1.95503610e-18, 1.01945860e-18,
       5.71980066e-19, 2.85902674e-19, 1.69509079e-19, 2.19433754e-34])

In [110]:
np.round(test_opt.symmetry_case.right_projected_symmetry_state.to_ndarray(), 2)

array([[-0.14+0.j,  0.27+0.j,  0.38+0.j, -0.08+0.j],
       [ 0.27+0.j,  0.38+0.j, -0.16+0.j,  0.06+0.j],
       [ 0.38+0.j, -0.16+0.j,  0.22+0.j, -0.17+0.j],
       [-0.08+0.j,  0.06+0.j, -0.17+0.j, -0.46+0.j]])

In [111]:
test_opt.add_new_layer()

In [112]:
test_opt.optimise_left_layer()

In [113]:
test_opt.optimise_right_layer()

In [114]:
test_opt.right_expectations

[[[[0.9999986344805548], [0.9999986344805548], [0.9999986344805548]]]]

In [115]:
np.round(test_opt.right_side_left_symmetry_environment.to_ndarray(), 2)

array([[-0.07+0.j,  0.14+0.j,  0.19+0.j, -0.04+0.j],
       [ 0.14+0.j,  0.19+0.j, -0.08+0.j,  0.03+0.j],
       [ 0.19+0.j, -0.08+0.j,  0.11+0.j, -0.09+0.j],
       [-0.04+0.j,  0.03+0.j, -0.09+0.j, -0.23+0.j]])

In [116]:
np.round(test_opt.symmetry_case.right_projected_symmetry_state.to_ndarray(), 2)

array([[-0.14+0.j,  0.27+0.j,  0.38+0.j, -0.08+0.j],
       [ 0.27+0.j,  0.38+0.j, -0.16+0.j,  0.06+0.j],
       [ 0.38+0.j, -0.16+0.j,  0.22+0.j, -0.17+0.j],
       [-0.08+0.j,  0.06+0.j, -0.17+0.j, -0.46+0.j]])

In [117]:
test_opt.right_side_left_symmetry_environment.to_ndarray()/test_opt.symmetry_case.right_projected_symmetry_state.to_ndarray()

array([[0.49999932-0.j, 0.49999932+0.j, 0.49999932+0.j, 0.49999932-0.j],
       [0.49999932+0.j, 0.49999932+0.j, 0.49999932-0.j, 0.49999932+0.j],
       [0.49999932+0.j, 0.49999932-0.j, 0.49999932+0.j, 0.49999932-0.j],
       [0.49999932-0.j, 0.49999932+0.j, 0.49999932-0.j, 0.49999932-0.j]])

So environments aren't the issue... check multiple layers.

In [118]:
test_case = cases[(True, (0, 0), 1, 3)]

In [119]:
block_width = 5
num_one_sided_iterations = 3
num_two_sided_iterations = 3
num_layers = 2
num_sites = 5

In [120]:
test_opt = MPSBrickSolver(
    test_case,
    num_sites,
    block_width,
    num_layers,
    num_one_sided_iterations,
    num_two_sided_iterations,
    max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM
)

In [121]:
test_opt.optimise()

In [122]:
exps = test_opt.flatten_exps()

In [123]:
np.round(exps, 2)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1.])

So that's also not an issue...

In [124]:
def contraction_calculate_right_side_left_environment(optimizer):
    opt = optimizer

    base_bs = opt.bottom_right_mps_tensors

    unitary = opt.right_unitaries[0][0].split_legs()

    t = unitary
    b=base_bs[0]

    t = npc.tensordot(
        t,
        b,
        [[f'p0*',], ['p',]]
    )

    t = npc.tensordot(
        t,
        b.conj(),
        [[f'p0',], ['p*',]]
    )

    for i, b in enumerate(base_bs[1:], start=1):
        t = npc.tensordot(
            t,
            b,
            [[f'p{i}*', 'vR'], ['p', 'vL']]
        )
    
        t = npc.tensordot(
            t,
            b.conj(),
            [[f'p{i}', 'vR*'], ['p*', 'vL*']]
        )
    
    t = npc.trace(t, 'vR', 'vR*')

    out = npc.tensordot(
        optimizer.symmetry_case.npc_symmetry_transfer_matrix,
        t,
        [['vR', 'vR*'], ['vL', 'vL*']]
    )

    return out

In [125]:
env = contraction_calculate_right_side_left_environment(test_opt)

In [126]:
env.to_ndarray()

array([[-0.00424838+0.j, -0.01686474+0.j, -0.09780925+0.j,
         0.22941422+0.j],
       [-0.01686474+0.j, -0.09241794+0.j,  0.21559566+0.j,
         0.08481165+0.j],
       [-0.09780925+0.j,  0.21559566+0.j,  0.06786302+0.j,
         0.04297059+0.j],
       [ 0.22941422+0.j,  0.08481165+0.j,  0.04297059+0.j,
         0.0288033 +0.j]])

In [127]:
test_opt.right_expectations

[[[[0.9999986344805548], [0.9999986344805548], [0.9999986344805548]],
  [[1.0000000000000004], [1.0000000000000004], [1.0000000000000004]],
  [[1.0000000000000007], [1.0000000000000007], [1.0000000000000007]]],
 [[[1.000000000000001], [1.000000000000001], [1.000000000000001]],
  [[1.0000000000000002], [1.0000000000000002], [1.0000000000000002]],
  [[1.0000000000000007], [1.0000000000000007], [1.0000000000000007]]]]

In [128]:
manual_exps = {
    k: v.manual_expectation_value()
    for k, v in optimizer_dict.items()
}

In [129]:
manual_exps

{(False, (1, 1), 0, 0): (1.0000000000000004+0j),
 (False, (1, 1), 0, 1): (1.0000000000000009+0j),
 (False, (1, 1), 0, 2): (1.0000000000000016+0j),
 (False, (1, 1), 0, 3): (1.0000000000000009+0j),
 (False, (1, 1), 1, 0): (1.000000000000002+0j),
 (False, (1, 1), 1, 1): (1.0000000000000018+0j),
 (False, (1, 1), 1, 2): (1.0000000000000013+0j),
 (False, (1, 1), 1, 3): (1.0000000000000013+0j),
 (True, (1, 1), 0, 0): (1.0000000000000129+0j),
 (True, (1, 1), 0, 1): (1.0000000000000127+0j),
 (True, (1, 1), 0, 2): (1.0000000000000138+0j),
 (True, (1, 1), 0, 3): (1.0000000000000129+0j),
 (True, (1, 1), 1, 0): (1.0000000000000127+0j),
 (True, (1, 1), 1, 1): (1.0000000000000133+0j),
 (True, (1, 1), 1, 2): (1.0000000000000138+0j),
 (True, (1, 1), 1, 3): (1.0000000000000129+0j),
 (True, (0, 1), 0, 0): (1.0000000000000135+0j),
 (True, (0, 1), 0, 1): (1.0000000000000133+0j),
 (True, (0, 1), 0, 2): (1.0000000000000138+0j),
 (True, (0, 1), 0, 3): (1.0000000000000127+0j),
 (True, (0, 1), 1, 0): (1.0000000

Same errors.

In [130]:
test_case = cases[(False, (1, 1), 0, 1)]

In [131]:
block_width = 5
num_one_sided_iterations = 3
num_two_sided_iterations = 3
num_layers = 1
num_sites = 5

In [132]:
test_opt = MPSBrickSolver(
    test_case,
    num_sites,
    block_width,
    num_layers,
    num_one_sided_iterations,
    num_two_sided_iterations,
    max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM
)

Manually test

In [133]:
test_opt.add_new_layer()

In [134]:
test_opt.optimise_left_layer()

In [135]:
test_opt.left_expectations

[[[[3.3718442430111543e-14],
   [3.3718442430111543e-14],
   [3.3718442430111543e-14]]]]

In [136]:
test_opt.manual_expectation_value()

(3.469446946573988e-18+0j)

In [137]:
test_opt.optimise_right_layer()

In [138]:
test_opt.right_expectations

[[[[0.9999997940932678], [0.9999997940932678], [0.9999997940932678]]]]

In [139]:
test_opt.manual_expectation_value()

(3.469446946573988e-18+0j)

In [140]:
test_opt.symmetry_case.symmetry_transfer_matrix_singular_vals

array([5.00000000e-01, 1.08870169e-17, 4.04910676e-18, 6.03379860e-33])

In [141]:
test_opt.right_unitaries

[]

In [142]:
test_opt.finish_current_layer()

In [143]:
test_opt.right_unitaries

[[<npc.Array shape=(256, 256) labels=['(p0.p1.p2.p3.p4)', '(p0*.p1*.p2*.p3*.p4*)']>]]

In [144]:
test_opt.manual_expectation_value()

(0.9999997940932606+0j)

In [145]:
block_width = 5
num_one_sided_iterations = 3
num_two_sided_iterations = 3
num_layers = 1
num_sites = 5

In [146]:
test_opt = MPSBrickSolver(
    test_case,
    num_sites,
    block_width,
    num_layers,
    num_one_sided_iterations,
    num_two_sided_iterations,
    max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM
)

In [147]:
test_opt.optimise()

In [148]:
test_opt.flatten_exps()

[3.3718442430111543e-14,
 3.3718442430111543e-14,
 3.3718442430111543e-14,
 3.3718442430111543e-14,
 3.3718442430111543e-14,
 3.3718442430111543e-14,
 3.3718442430111543e-14,
 3.3718442430111543e-14,
 3.3718442430111543e-14,
 0.9999997940932678,
 0.9999997940932678,
 0.9999997940932678,
 0.9999997940932678,
 0.9999997940932678,
 0.9999997940932678,
 0.9999997940932678,
 0.9999997940932678,
 0.9999997940932678,
 1.0000000000000042,
 1.0000000000000042,
 1.0000000000000042,
 1.0000000000000042,
 1.0000000000000042,
 1.0000000000000042,
 1.0000000000000042,
 1.0000000000000042,
 1.0000000000000042,
 1.0000000000000069,
 1.0000000000000069,
 1.0000000000000069,
 1.0000000000000069,
 1.0000000000000069,
 1.0000000000000069,
 1.0000000000000069,
 1.0000000000000069,
 1.0000000000000069,
 1.000000000000004,
 1.000000000000004,
 1.000000000000004,
 1.000000000000004,
 1.000000000000004,
 1.000000000000004,
 1.000000000000004,
 1.000000000000004,
 1.000000000000004,
 1.0000000000000073,
 1.0000

In [149]:
test_opt.manual_expectation_value()

(1.0000000000000009+0j)

In [150]:
test_opt.right_unitaries

[[<npc.Array shape=(256, 256) labels=['(p0.p1.p2.p3.p4)', '(p0*.p1*.p2*.p3*.p4*)']>]]

In [151]:
block_width = 5
num_one_sided_iterations = 3
num_two_sided_iterations = 1
num_layers = 1
num_sites = 5

In [152]:
test_opt = MPSBrickSolver(
    test_case,
    num_sites,
    block_width,
    num_layers,
    num_one_sided_iterations,
    num_two_sided_iterations,
    max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM
)

In [153]:
test_opt.optimise()

In [154]:
test_opt.flatten_exps()

[3.3718442430111543e-14,
 3.3718442430111543e-14,
 3.3718442430111543e-14,
 3.3718442430111543e-14,
 3.3718442430111543e-14,
 3.3718442430111543e-14,
 3.3718442430111543e-14,
 3.3718442430111543e-14,
 3.3718442430111543e-14,
 0.9999997940932678,
 0.9999997940932678,
 0.9999997940932678,
 0.9999997940932678,
 0.9999997940932678,
 0.9999997940932678,
 0.9999997940932678,
 0.9999997940932678,
 0.9999997940932678]

In [155]:
test_opt.manual_expectation_value()

(0.9999997940932606+0j)

In [156]:
block_width = 5
num_one_sided_iterations = 1
num_two_sided_iterations = 3
num_layers = 1
num_sites = 5

In [157]:
test_opt = MPSBrickSolver(
    test_case,
    num_sites,
    block_width,
    num_layers,
    num_one_sided_iterations,
    num_two_sided_iterations,
    max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM
)

In [158]:
test_opt.optimise()

In [159]:
test_opt.flatten_exps()

[3.3718442430111543e-14,
 0.9999997940932678,
 1.0000000000000042,
 1.0000000000000069,
 1.000000000000004,
 1.0000000000000073]

In [160]:
test_opt.manual_expectation_value()

(1.0000000000000009+0j)